Файл для тренеровки нейронной сети на данных с окном в 150 точек

In [3]:
import os 
import glob
import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from sklearn.externals import joblib

/home/andr/.pyenv/versions/3.7.4/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [5]:
# Считываем набор файлов из типизированных директорий
zero_files = [os.path.basename(i)[:-4] for i in glob.glob('./results_set/typizing/0/*')]
one_files = [os.path.basename(i)[:-4] for i in glob.glob('./results_set/typizing/1/*')]

In [6]:
dataset_pulse = pd.DataFrame(
    columns=[f'point_{i}' for i in range(150)]
)
dataset_pulse['type'] = None

In [7]:
idx = 0
# добавляем импульсы
for name in one_files:
    profile = np.genfromtxt(f'./results_set/file_untypized/{name}.csv', skip_header=6)
    profile /= max(profile)  # Нормировка для выделения только формы импульса
    try:
        dataset_pulse.loc[idx] = [
            *profile,
            1
        ]
        idx += 1
    except ValueError:  # Избегаем импульсов короче 150 точек
        continue
# Добавляем помехи
for name in zero_files:
    profile = np.genfromtxt(f'./results_set/file_untypized/{name}.csv', skip_header=6)
    profile /= max(profile)  # Нормировка для выделения только формы импульса
    try:
        dataset_pulse.loc[idx] = [
            *profile,
            0
        ]
        idx += 1
    except ValueError:  # Избегаем импульсов короче 150 точек
        continue

In [8]:
X = dataset_pulse.drop(['type'], axis=1)
y = dataset_pulse['type']

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [10]:
clf = RandomForestClassifier(n_estimators=1000)
clf = clf.fit(X_train, y_train)
predictions = clf.predict(X_test)
print(accuracy_score(predictions, y_test))

0.986328125


In [4]:
NOW = datetime.datetime.now().strftime("%Y-%m-%d")

In [30]:
# Сохранение базы данных
dataset_pulse.to_csv(f'typized_dataset_for_dina_{NOW}.csv', sep='\t', header=True, index=False)

In [36]:
# Сохранение обученной нейросети
filename = 'dina_model_RFC(1000)_pulse_noise_95_{NOW}.sav'
joblib.dump(clf, filename)

['dina_model_RFC(1000)_pulse_noise_95.sav']